In [5]:
import csv
import json
import sqlite3


calculation_type = 'drop_training' # exclude training samples when gather human results 
keep_catch_trials = False 

results = {'inlab':[], 'inlab_gender':[], 'inlab_nogender':[], 'mturk':[], 'machine':[], 'combined':[], 'mturk-feedback':[]}

############# for jason file processing
jsonpath = './Data_all/'


# if calculation_type == 'drop_training':
#     with open(jsonpath+"feedback_train_samples.json", "r") as fp:
#         feedback_train_samples = json.load(fp)["feedback_train_samples"]
#     with open(jsonpath+"feedback_test_samples.json", "r") as fp:
#         feedback_test_samples = json.load(fp)["feedback_test_samples"]        
# filenames = {#'inlab': ['turing_test_img_caption_inlab_gender_15','turing_test_img_caption_inlab_no_gender_15'],
#                  #'inlab_gender': ['turing_test_img_caption_inlab_gender_15'],
#                  #'inlab_nogender': ['turing_test_img_caption_inlab_no_gender_15'],
#                  #'mturk':['turing_live_mturk'],
#                  'mturk-feedback':['turing_live_feedback']}

filenames = {}


#Images 1-250 are from COCO test set, 
# images 251-500 are nocaps val in-domain, 
# images 501-750 are nocaps val near-domain, 
# images 751-1000 are nocaps val out-domain



In [6]:

for key in filenames:
    print("=======================")
    print(key)

    count = 0
    trials_dict = {}
    workercount = 0
    
    
    for dbfilename in filenames[key]:
        #print("=======================")
        #print(jasonfilename)
        conn = sqlite3.connect(jsonpath + dbfilename + ".db")
        cursor = conn.cursor()
        cursor.execute("select * from imgcaption;")        
        all_data = cursor.fetchall()

        for i in range(0, len(all_data)):
            nTrial = 0
            if (all_data[i][15] == 7) or (all_data[i][15] == 5): #check status 
                data_dict = json.loads(all_data[i][17])
                workercount += 1
                for dat in data_dict['data']:
                    if dat['trialdata']['phase'] == 'TEST':
                        nTrial += 1
#                         print(dat)
                        if calculation_type == "drop_training" and key == "mturk-feedback" and\
                            nTrial<=40:
#                             feedback_train_samples.append()
                            continue
#                         elif calculation_type == "drop_training" and key != "mturk-feedback" and\
#                             [dat['trialdata']['imageID'].split('/')[-1],\
#                              dat['trialdata']['human_grountruth'],\
#                              dat['trialdata']['ai_grountruth']] not in feedback_test_samples:
#                             train_samples.append([dat['trialdata']['imageID'].split('/')[-1],\
#                              dat['trialdata']['human_grountruth'],\
#                              dat['trialdata']['ai_grountruth']])
#                             continue
                        elif key == "mturk-feedback" and not keep_catch_trials and dat['trialdata']['machine_groundtruth'] == "catch":
                            continue

                        count += 1
                        trial_data = {}
                        trial_data['workerID'] = data_dict['workerId']
                        trial_data['workerData'] = data_dict['questiondata']
                        trial_data['trialData'] = dat['trialdata']
                        
                        imagepathname_id = int(trial_data['trialData']['imageID'][-8:-4])
                    
                        if imagepathname_id<= 250:
                            trial_data['workerData']['imagetype'] = 'COCO'
                        elif imagepathname_id<= 500:
                            trial_data['workerData']['imagetype'] = 'in-domain'
                        elif imagepathname_id<= 750:
                            trial_data['workerData']['imagetype'] = 'near-domain'
                        else:
                            trial_data['workerData']['imagetype'] = 'out-domain'
                        
                        
                        # Get rid of typo in the key in db file
                        if 'ai_grountruth' in list(trial_data['trialData'].keys()):
                            trial_data['trialData']['machine_groundtruth'] = trial_data['trialData']['ai_grountruth']
                            del trial_data['trialData']['ai_grountruth']

                        # Get rid of typo in the key in db file
                        if trial_data['trialData']['response_speaker'] == 'ai':
                            trial_data['trialData']['response_speaker'] = 'machine'

                        trials_dict['Trial_'+str(count)] = trial_data

                
        conn.close()
        results[key] = trials_dict
        #json.dump(trials_dict, open("./turing_inlab_no_gender_img_caption_half.json","w"))
    print(workercount, ' workers')

# ########### AI jason files
# json_file_path = './Data_all/ai_judge_freeviewing_feedback.json'

# with open(json_file_path, 'r') as j:
#      contents = json.loads(j.read())
# results['machine-feedback'] = contents
# #print(contents)
# json_file_path = './Data_all/ai_judge_zeroshot_chatgpt_fv.json'

# with open(json_file_path, 'r') as j:
#      contents = json.loads(j.read())
# results['machine-zeroshot'] = contents
# #save combined results
# count = 0
# trials_dict = {}
# for ke in ['mturk','mturk-feedback']:
#     for it in results[ke]:
#         count = count + 1
#         trialin = results[ke][it]
#         trials_dict['Trial_'+str(count)] = trialin
        
# results['combined'] = trials_dict

json_file_path = './Data_all/complete_humanjudge_VS_Ablated.json'

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())
results['mturk-vs-ablated'] = contents

# with open("./Data_all/humanjudge_FV_Feedback_testphase.json",'r') as f:
#     contents = json.loads(f.read())

# results["mturk-feedback"] = contents 



print('All data is extracted.')
for key in results:
    print(key)
    print('There are [' + str(len(results[key])) + ']')
    
#save the compiled results as jason
with open("./savedResults/compiled_VS_Ablated.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

All data is extracted.
inlab
There are [0]
inlab_gender
There are [0]
inlab_nogender
There are [0]
mturk
There are [0]
machine
There are [0]
combined
There are [0]
mturk-feedback
There are [0]
mturk-vs-ablated
There are [1920]
Compilation completed! Results are saved to folder!


Bad pipe message: %s [b'\xe7\x80K"\xdc\xc8\x8f\xae\xba6\x11\xe1\xb2\x14e"\x98$\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b']
Bad pipe message: %s [b'\xf7\xd8\xf1M\x9c\xb6\x9b\xc3\x83x\xf9\x0b\x06\xc7\xa3\xa2!\xa4\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00', b'\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#']
Bad pipe message: %s [b'\xa0\x93W\x1d\xc4\xbc\x1c\xde:0', b'4]\xa0@_m\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\